# Merin Joseph
## CPSC 597-project: i-Grader ( Faculty Assist System: AI Detection)
### CSUF - CS Grad
### CWID: 885869974

In [30]:
import pandas as pd
from sklearn.metrics import accuracy_score
import pickle
import torch
from transformers import pipeline
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification, AdamW


In [31]:
pipe = pipeline("text-classification", model="roberta-base")

C:\Users\ponny\.conda\envs\aiproj\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ponny\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: [

In [32]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForMaskedLM.from_pretrained("roberta-base")

In [33]:
data_test = '''Laughter is a universal language that transcends cultural, linguistic, and social barriers. It is a powerful and instinctive human expression that brings joy, connection, and numerous health benefits. The importance of laughter in our lives cannot be overstated, as it plays a pivotal role in enhancing our physical, mental, and emotional well-being.On a physiological level, laughter triggers a cascade of positive effects within the body. It promotes the release of endorphins, often referred to as "feel-good" hormones, which contribute to a sense of happiness and euphoria. The act of laughing also stimulates the cardiovascular system, increasing blood flow and improving heart health. Additionally, laughter has been shown to boost the immune system, providing a natural defense against illness and stress.'''

In [34]:
data_test_human='''Laughter for me is very important as it is what hold my family together. we have our qaulity family time everyday at night where we share what happened in our day and so on. This makes me so happy. I love my family.'''

In [35]:
file_path = r"C:\Users\ponny\Desktop\final_proj\wiki-labeled.csv"  
df = pd.read_csv(file_path)

In [39]:
texts = df['text'].tolist()[:30000]
labels = df['label'].tolist()[:30000]

In [43]:
tokenizer = AutoTokenizer.from_pretrained("andreas122001/roberta-wiki-detector")

In [44]:
tokenized_data = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
# outputs = model(**inputs)

In [50]:
train_inputs, val_inputs, train_labels, val_labels = train_test_split(tokenized_data['input_ids'], labels, test_size=0.2, random_state=42)

In [51]:
optimizer = AdamW(model.parameters(), lr=5e-5)

C:\Users\ponny\.conda\envs\aiproj\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [57]:
print(train_inputs[1])

tensor([    0, 41532, 23566,    16,    10,  9095,   470,   156,    12,  1990,
           12,   859, 41605,  4003, 25032,  4149,   822,  3660,    30,   610,
         4594,   397,     8,  1982,    30,  2897,  9909,     8,  7896,    91,
         2871,     4,    85,  2690,    91,  2871,    25, 11882, 23566,  3199,
         2094,     8,  4594,   397,    25,    69,  1623,     6,     5,  3395,
          748,  5247,  3623,  4061, 12576,     8,  3331,     6,  2101,  3643,
          448,  7165,  1020,     4, 50118, 50118,   133,   822,  3026,     5,
          527,     9, 11882, 23566,  3199,  2094,     6,    41,   470,   311,
        19347,    54,  1458,     7,  9444,    11,     5, 18283,    29,     8,
        15891,    29,     4,  1405,  7116,   301,   111,   455,     9,  1289,
        10121,    29,     8,  1081,  6906,   111,    16, 21994,   149, 37986,
            6,    25,    79,  3026,    69,   527,     7,    69,   251,    12,
        30403,  2961,  1623,  2101,  3643,   448,  7165,  1020, 

In [66]:
# Fine-tune the model
model.train()
for epoch in range(3):  
    optimizer.zero_grad()
    
    outputs = model(train_inputs, labels=train_labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()


KeyboardInterrupt



In [ ]:
# Save the tokenizer and model using pickle

pickle.dump(tokenizer, open(r"C:\Users\ponny\Desktop\final_proj\webapp\aitokenizer.pkl", 'wb'))
pickle.dump(model,open(r"C:\Users\ponny\Desktop\final_proj\webapp\aimodel.pkl", 'wb'))

# Load the tokenizer and model back using pickle
loaded_tokenizer = pickle.load(open(r"C:\Users\ponny\Desktop\final_proj\webapp\aitokenizer.pkl", 'rb'))
loaded_model = pickle.load(open(r"C:\Users\ponny\Desktop\final_proj\webapp\aimodel.pkl", 'rb'))

In [28]:
if predicted_class == 1:
    print("The sample text is AI-generated or AI-related.")
else:
    print("The sample text is human-authored or not AI-related.")

The sample text is AI-generated or AI-related.


In [ ]:
df_subset = df.head(2000)

In [ ]:
texts = df_subset['text'].tolist()

In [ ]:
true_labels = df_subset['label'].tolist()

In [ ]:
predicted_labels = []

In [ ]:
print(len(texts))

In [22]:
for i in range(len(texts)):
    text=texts[i]
    true_label=true_labels[i]
    # Tokenize the text
    inputs = loaded_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    
    # Make predictions
    outputs = loaded_model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    
    # Append the predicted label to the list
    predicted_labels.append(predicted_class)
    print(f"Processing row {i + 1}: Predicted class - {predicted_class}, True label - {true_label}")

Processing row 1: Predicted class - 0, True label - 0
Processing row 2: Predicted class - 1, True label - 1
Processing row 3: Predicted class - 0, True label - 0
Processing row 4: Predicted class - 1, True label - 1
Processing row 5: Predicted class - 0, True label - 0
Processing row 6: Predicted class - 1, True label - 1
Processing row 7: Predicted class - 0, True label - 0
Processing row 8: Predicted class - 1, True label - 1
Processing row 9: Predicted class - 0, True label - 0
Processing row 10: Predicted class - 1, True label - 1
Processing row 11: Predicted class - 0, True label - 0
Processing row 12: Predicted class - 1, True label - 1
Processing row 13: Predicted class - 0, True label - 0
Processing row 14: Predicted class - 1, True label - 1
Processing row 15: Predicted class - 0, True label - 0
Processing row 16: Predicted class - 1, True label - 1
Processing row 17: Predicted class - 0, True label - 0
Processing row 18: Predicted class - 1, True label - 1
Processing row 19: 

Processing row 150: Predicted class - 1, True label - 1
Processing row 151: Predicted class - 0, True label - 0
Processing row 152: Predicted class - 1, True label - 1
Processing row 153: Predicted class - 0, True label - 0
Processing row 154: Predicted class - 1, True label - 1
Processing row 155: Predicted class - 0, True label - 0
Processing row 156: Predicted class - 1, True label - 1
Processing row 157: Predicted class - 0, True label - 0
Processing row 158: Predicted class - 1, True label - 1
Processing row 159: Predicted class - 0, True label - 0
Processing row 160: Predicted class - 1, True label - 1
Processing row 161: Predicted class - 0, True label - 0
Processing row 162: Predicted class - 1, True label - 1
Processing row 163: Predicted class - 0, True label - 0
Processing row 164: Predicted class - 1, True label - 1
Processing row 165: Predicted class - 0, True label - 0
Processing row 166: Predicted class - 1, True label - 1
Processing row 167: Predicted class - 0, True la

Processing row 297: Predicted class - 0, True label - 0
Processing row 298: Predicted class - 1, True label - 1
Processing row 299: Predicted class - 0, True label - 0
Processing row 300: Predicted class - 1, True label - 1
Processing row 301: Predicted class - 0, True label - 0
Processing row 302: Predicted class - 1, True label - 1
Processing row 303: Predicted class - 0, True label - 0
Processing row 304: Predicted class - 1, True label - 1
Processing row 305: Predicted class - 0, True label - 0
Processing row 306: Predicted class - 1, True label - 1
Processing row 307: Predicted class - 0, True label - 0
Processing row 308: Predicted class - 1, True label - 1
Processing row 309: Predicted class - 0, True label - 0
Processing row 310: Predicted class - 1, True label - 1
Processing row 311: Predicted class - 0, True label - 0
Processing row 312: Predicted class - 1, True label - 1
Processing row 313: Predicted class - 0, True label - 0
Processing row 314: Predicted class - 1, True la

Processing row 444: Predicted class - 1, True label - 1
Processing row 445: Predicted class - 0, True label - 0
Processing row 446: Predicted class - 1, True label - 1
Processing row 447: Predicted class - 0, True label - 0
Processing row 448: Predicted class - 1, True label - 1
Processing row 449: Predicted class - 0, True label - 0
Processing row 450: Predicted class - 1, True label - 1
Processing row 451: Predicted class - 0, True label - 0
Processing row 452: Predicted class - 1, True label - 1
Processing row 453: Predicted class - 0, True label - 0
Processing row 454: Predicted class - 1, True label - 1
Processing row 455: Predicted class - 0, True label - 0
Processing row 456: Predicted class - 1, True label - 1
Processing row 457: Predicted class - 0, True label - 0
Processing row 458: Predicted class - 1, True label - 1
Processing row 459: Predicted class - 0, True label - 0
Processing row 460: Predicted class - 1, True label - 1
Processing row 461: Predicted class - 0, True la

Processing row 591: Predicted class - 0, True label - 0
Processing row 592: Predicted class - 1, True label - 1
Processing row 593: Predicted class - 0, True label - 0
Processing row 594: Predicted class - 1, True label - 1
Processing row 595: Predicted class - 0, True label - 0
Processing row 596: Predicted class - 1, True label - 1
Processing row 597: Predicted class - 0, True label - 0
Processing row 598: Predicted class - 1, True label - 1
Processing row 599: Predicted class - 0, True label - 0
Processing row 600: Predicted class - 1, True label - 1
Processing row 601: Predicted class - 0, True label - 0
Processing row 602: Predicted class - 1, True label - 1
Processing row 603: Predicted class - 0, True label - 0
Processing row 604: Predicted class - 1, True label - 1
Processing row 605: Predicted class - 0, True label - 0
Processing row 606: Predicted class - 1, True label - 1
Processing row 607: Predicted class - 0, True label - 0
Processing row 608: Predicted class - 1, True la

...

Processing row 1319: Predicted class - 0, True label - 0
Processing row 1320: Predicted class - 1, True label - 1
Processing row 1321: Predicted class - 0, True label - 0
Processing row 1322: Predicted class - 1, True label - 1
Processing row 1323: Predicted class - 0, True label - 0
Processing row 1324: Predicted class - 1, True label - 1
Processing row 1325: Predicted class - 0, True label - 0
Processing row 1326: Predicted class - 1, True label - 1
Processing row 1327: Predicted class - 0, True label - 0
Processing row 1328: Predicted class - 1, True label - 1
Processing row 1329: Predicted class - 0, True label - 0
Processing row 1330: Predicted class - 1, True label - 1
Processing row 1331: Predicted class - 0, True label - 0
Processing row 1332: Predicted class - 1, True label - 1
Processing row 1333: Predicted class - 0, True label - 0
Processing row 1334: Predicted class - 1, True label - 1
Processing row 1335: Predicted class - 0, True label - 0
Processing row 1336: Predicted 

Processing row 1463: Predicted class - 0, True label - 0
Processing row 1464: Predicted class - 1, True label - 1
Processing row 1465: Predicted class - 0, True label - 0
Processing row 1466: Predicted class - 1, True label - 1
Processing row 1467: Predicted class - 0, True label - 0
Processing row 1468: Predicted class - 1, True label - 1
Processing row 1469: Predicted class - 0, True label - 0
Processing row 1470: Predicted class - 1, True label - 1
Processing row 1471: Predicted class - 0, True label - 0
Processing row 1472: Predicted class - 1, True label - 1
Processing row 1473: Predicted class - 0, True label - 0
Processing row 1474: Predicted class - 1, True label - 1
Processing row 1475: Predicted class - 0, True label - 0
Processing row 1476: Predicted class - 1, True label - 1
Processing row 1477: Predicted class - 0, True label - 0
Processing row 1478: Predicted class - 1, True label - 1
Processing row 1479: Predicted class - 0, True label - 0
Processing row 1480: Predicted 

Processing row 1607: Predicted class - 0, True label - 0
Processing row 1608: Predicted class - 1, True label - 1
Processing row 1609: Predicted class - 0, True label - 0
Processing row 1610: Predicted class - 1, True label - 1
Processing row 1611: Predicted class - 0, True label - 0
Processing row 1612: Predicted class - 1, True label - 1
Processing row 1613: Predicted class - 0, True label - 0
Processing row 1614: Predicted class - 1, True label - 1
Processing row 1615: Predicted class - 0, True label - 0
Processing row 1616: Predicted class - 1, True label - 1
Processing row 1617: Predicted class - 0, True label - 0
Processing row 1618: Predicted class - 1, True label - 1
Processing row 1619: Predicted class - 0, True label - 0
Processing row 1620: Predicted class - 1, True label - 1
Processing row 1621: Predicted class - 0, True label - 0
Processing row 1622: Predicted class - 1, True label - 1
Processing row 1623: Predicted class - 0, True label - 0
Processing row 1624: Predicted 

Processing row 1751: Predicted class - 0, True label - 0
Processing row 1752: Predicted class - 1, True label - 1
Processing row 1753: Predicted class - 0, True label - 0
Processing row 1754: Predicted class - 1, True label - 1
Processing row 1755: Predicted class - 0, True label - 0
Processing row 1756: Predicted class - 1, True label - 1
Processing row 1757: Predicted class - 0, True label - 0
Processing row 1758: Predicted class - 1, True label - 1
Processing row 1759: Predicted class - 0, True label - 0
Processing row 1760: Predicted class - 1, True label - 1
Processing row 1761: Predicted class - 0, True label - 0
Processing row 1762: Predicted class - 1, True label - 1
Processing row 1763: Predicted class - 0, True label - 0
Processing row 1764: Predicted class - 1, True label - 1
Processing row 1765: Predicted class - 0, True label - 0
Processing row 1766: Predicted class - 1, True label - 1
Processing row 1767: Predicted class - 0, True label - 0
Processing row 1768: Predicted 

Processing row 1895: Predicted class - 0, True label - 0
Processing row 1896: Predicted class - 1, True label - 1
Processing row 1897: Predicted class - 0, True label - 0
Processing row 1898: Predicted class - 1, True label - 1
Processing row 1899: Predicted class - 0, True label - 0
Processing row 1900: Predicted class - 1, True label - 1
Processing row 1901: Predicted class - 0, True label - 0
Processing row 1902: Predicted class - 1, True label - 1
Processing row 1903: Predicted class - 0, True label - 0
Processing row 1904: Predicted class - 1, True label - 1
Processing row 1905: Predicted class - 0, True label - 0
Processing row 1906: Predicted class - 1, True label - 1
Processing row 1907: Predicted class - 0, True label - 0
Processing row 1908: Predicted class - 1, True label - 1
Processing row 1909: Predicted class - 0, True label - 0
Processing row 1910: Predicted class - 1, True label - 1
Processing row 1911: Predicted class - 0, True label - 0
Processing row 1912: Predicted 

In [28]:
accuracy = accuracy_score(true_labels, predicted_labels)



In [29]:
print(f'Accuracy on the first 2000 rows: {accuracy*100}')

Accuracy on the first 2000 rows: 99.85000000000001
